In [75]:
import numpy as np

def calcuLog(a,b):
    try:
        result = np.log(float(a)/float(b))
    except:
        print "divided by 0"
    return result

def createNgramDict(train_list):
    trainDict = {}
    train_key = ' '.join(train_list[2:])
    train_value = train_list[0]
    return train_key,train_value

#######cannot add trainDict1[train_key] = int(train_value) and return the whole dict because dict is stillbuiding 
####first buiding three dictionaries for 1 gram,bigram and trigram for faster speed ###################
def buildDictionary(input_file):
    try:
        train_data = open(input_file,'r')
    except IOException as err:
        print "no such file"
    trainDict1 = {}
    trainDict2 = {}
    trainDict3 = {}
    for train_sentence in train_data:
        train_list = train_sentence.strip().split(" ")
        if train_list[1] =="1-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict1[train_key] = int(train_value)
        elif train_list[1] == "2-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict2[train_key] = int(train_value)
        elif train_list[1] == "3-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict3[train_key] = int(train_value)
    return trainDict1,trainDict2,trainDict3
#####the q parameter is calculated by trigram divided by bigram########################################

def calQparameter(trigram_each):
    trigram = trigram_each.strip()
    bigram = ' '.join(trigram.split(" ")[0:2])
    try:
        count_tri = trainDict3[trigram]
    except KeyError:
        count_tri = 0
    try:
        count_bi = trainDict2[bigram]
    except KeyError:
        count_bi = 0
    if count_bi != 0:
        result = calcuLog(count_tri,count_bi)
    else:
        result = float('-inf')
    return result
    #print output
    
def cal_emission(count_pair,count_y):
    try:
        emission_para = (float(count_pair)/float(count_y))
    except:
        pass #return "divide by zero"
    return emission_para

#this function finds the tag and the times it appears and save it in a dictionary
def findTagDict(count_file):
    #dictTag = {}#dict()
    try:
        count_content = open(count_file)
    except:
        print "no such file"
    for count_line in count_content:
        count_list = count_line.strip().split(" ")
        if count_list[1] == "1-GRAM":
            if count_list[-1] not in dictTag:
                dictTag[count_list[-1]] = int(count_list[0])
            else:
                dictTag[count_list[-1]] += int(count_list[0])
    #print dictTag
    return dictTag

#this functin finds the word and the frequency of a word tagged aby a specific tag
def findWordDict(count_file):
    dictWord = {}#dict() or will pop dict() not callable error
    try:
        count_content = open(count_file)
    except:
        print "no such file"
    for count_line in count_content:
        count_list = count_line.strip().split(" ")
        word = count_list[-1]
        tag = count_list[2]
        number = count_list[0]
        if count_list[1] =="WORDTAG":
            #initialize
            if word not in dictWord:
                dictWord[word] = {}  
            #add element
            if tag not in dictWord[word]:
                dictWord[word][tag] = int(number)
            else:
                dictWord[word][tag] += int(number)
    for word in dictWord:
        max_ratio = 0
        max_tag = ""
        for tag in dictWord[word]:
            dictWord[word][tag] = np.log2(cal_emission(dictWord[word][tag],dictTag[tag]))
    #print dictWord        
    return dictWord

def findEmission(word_input,tag_input):
    '''for word in dictWord:
        max_ratio = 0
        max_tag = ""
        for tag in dictWord[word]:
            dictWord[word][tag] = np.log2(cal_emission(dictWord[word][tag],dictTag[tag]))
    #return dictWord'''
    if word_input.strip() not in dictWord:
        original_word = word_input.strip()
        try:
            value = dictWord["_RARE_"][tag_input]
        except:
            value = float("-inf")
    else:#lif test_line.strip() in dictMax://cannot use word and tag because they are the lastelement itmes keys and values
        original_word = word_input.strip()
        try:
            value = dictWord[original_word][tag_input]
        except:
            value = float("-inf")
    return value


           

In [76]:
def Viterbi(sentence):
    #initialization
    n = len(sentence)
    print n
    tag_sequence = []
    tag_value = []
    pI = {}
    K_n1 = ['*']
    K_0 = ['*']
    K = ['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
    #keys = str(position)","+u+","+v
    pI['0,*,*'] = 1# only store the maximum value corresponding to each position given u and v
    bp = {}
    #########################the first step of Viterbi##################
    for k in range(1,n+1):
        Vk = K
        if k == 1:
            Uk = K_0
            Wk = K_n1    
        elif k == 2:
            Uk = K
            Wk = K_0
        else:
            Uk = K
            Wk = K
        print Uk
        print Wk
        print Vk
        print "#######"
        for u in Uk:
            for v in Vk:
                optimal_val = float("-inf")
                optimal_w = ''
                #make u v as given condition,change the value of Wk
                for w in Wk:
                    trigram_each = str(w)+" "+str(u)+" "+str(v)
                    q = calQparameter(trigram_each)
                    word_input = sentence[k-1]
                    tag_input = str(v)
                    '''print word_input
                    print tag_input'''
                    e = findEmission(word_input,tag_input) 
                    key_last = str(k-1)+","+str(w)+","+str(u)
                    '''print pI[key_last]
                    print q
                    print e'''
                    tmp = float(pI[key_last]) + q + e
                    #print tmp
                    if tmp > optimal_val:
                        optimal_val = tmp
                        optimal_w = w
                    else:
                        optimal_val = optimal_val
                        optimal_w = optimal_w
                '''print "the optimal value is "+str(optimal_val)
                print "v:"+str(v)+"u:"+str(u)'''
                key = str(k)+","+str(u)+","+str(v)
                pI[key] = optimal_val
                bp[key] = optimal_w
    
    ##########################step 2 of Viterbi Algorithm######################
    max_u = "haha"
    max_v = "hahah"
    max_value = float("-inf")
    for u in K:
        for v in K:
            trigram_each = str(u)+" "+str(v)+" "+"STOP"
            q = calQparameter(trigram_each)
            key_fin = str(n)+","+str(u)+","+str(v)
            temp = pI[key_fin] + q
            if temp > max_value:
                max_value = temp
                max_u = u
                max_v = v
    print pI
    print bp
    tag_sequence.insert(0,max_v)
    tag_sequence.insert(0,max_u)
    print tag_sequence
    ######################step 3 of Viterbi Algorithm#########################
    #insert all y sequence backwards
    #insert n~2 sequence of pi values backwards
    for k in range(n-2,0,-1):
        key_bp = str(k+2)+","+str(tag_sequence[0])+","+str(tag_sequence[1])
        tag_value.insert(0,pI[key_bp])
        y_k = bp[key_bp]
        tag_sequence.insert(0,y_k)
        #print len(tag_sequence)
    #insert the second value of pi
    key1 = str(2)+","+str(tag_sequence[0])+","+str(tag_sequence[1])
    tag_value.insert(0,pI[key1])
    #insert the first value of pi
    key0 = str(1)+","+"*"+","+str(tag_sequence[0])
    tag_value.insert(0,pI[key0])
    print pI["1,*,O"]
    print tag_value
    print tag_sequence
    return tag_sequence,tag_value
                

In [77]:
if __name__ == "__main__":
    input_file = "ner_new.counts"   
    #testcase
    trainDict1,trainDict2,trainDict3 = buildDictionary(input_file)
    #testcase
    trigram_each = "* * B-ORG"
    result = calQparameter(trigram_each)
    print result
    test_input = "ner_dev.dat"
    dictTag = {}
    dictTag = findTagDict(input_file)
    dictWord = {}
    dictWord = findWordDict(input_file)
    #print dictWord
    #testcase
    '''word_input = "CRICKET"
    tag_input = "I-MISC"
    #value = findEmission(dictWord,dictTag,word_input,tag_input)
    value = findEmission(word_input,tag_input)
    print value'''
    sentence = ['CRICKET','-','LEICESTERSHIRE']
    tag_sequence,tag_value = Viterbi(sentence)
    

-inf
3
['*']
['*']
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
#######
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
['*']
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
#######
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
#######
{'3,B-LOC,I-MISC': -inf, '3,B-LOC,O': -inf, '3,B-LOC,B-LOC': -inf, '3,I-MISC,B-LOC': -inf, '3,I-LOC,B-MISC': -inf, '3,B-MISC,I-ORG': -inf, '2,I-MISC,I-LOC': -inf, '3,B-ORG,I-LOC': -inf, '2,B-ORG,I-LOC': -inf, '3,O,B-ORG': -inf, '3,I-PER,B-LOC': -inf, '2,I-LOC,B-ORG': -inf, '2,I-ORG,B-ORG': -inf, '2,I-MISC,O': -inf, '1,*,B-MISC': -inf, '3,I-LOC,I-LOC': -inf, '2,B-ORG,I-MISC': -inf, '2,I-ORG,O': -inf, '0,*,*': 1, '3,I-ORG,I-MISC': -inf, '3,I-ORG,I-PER': -inf, '3,B-ORG,I-PER': -inf, '2,B-LOC,B-MISC': -inf, '3,O,I-ORG': -24

/Users/yt2583columbia.edu/anaconda3/envs/Python27/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


In [ ]:
print dictTag

In [ ]:
a= 's'
str(a)

In [ ]:
lis1 = [1,2]
lis1.insert(0,4)
lis1

In [ ]:
list2 = [3,4,5]
li3 = zip(lis1,list2)
print type(li3[1])

In [ ]:
probs = [-1e10] * 10
print probs